## **Tarea 3**

In [ ]:
# Importación de todo lo necesario
import torch
import torch.nn as nn
from torchvision import datasets, transforms
import hydra
import pytorch_lightning as pl
from omegaconf import DictConfig
import matplotlib.pyplot as plt


transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

mnist_data = datasets.MNIST(root='./data', train=True, download=True, transform=transform)

# Definición del Autoencoder
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        
        # Encoder
        self.encoder = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=16, out_channels=8, kernel_size=3, stride=2, padding=1),
            nn.ReLU()
        )
        
        # Decoder
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(in_channels=8, out_channels=16, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(in_channels=16, out_channels=1, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.Sigmoid()  # Sigmoid para restringir los valores entre 0 y 1
        )
    
    def forward(self, x):
        encoded = self.encoder(x)  # Codificación de la imagen
        decoded = self.decoder(encoded)  # Reconstrucción de la imagen desde la codificación latente
        return decoded
    
# Definición de LightningModule
class LitAutoencoder(pl.LightningModule):
    def __init__(self, config):
        super(LitAutoencoder, self).__init__()
        self.learning_rate = config.model.learning_rate
        self.autoencoder = Autoencoder()

    def forward(self, x):
        return self.autoencoder(x)

    def training_step(self, batch):
        # Extraer las imágenes del batch
        x, _ = batch

        # Pasarlas por el autoencoder
        x_hat = self(x)

        # Calcular la pérdida (error cuadrático medio entre original y reconstruido)
        loss = nn.functional.mse_loss(x_hat, x)

        # Registrar el valor de la pérdida
        self.log('train_loss', loss)
        return loss
    
    def test_step(self, batch):
        # Extraer las imágenes del batch
        x, _ = batch

        # Reconstruir las imágenes
        x_hat = self(x)

        # Calcular la pérdida de prueba
        loss = nn.functional.mse_loss(x_hat, x)

        # Registrar el valor de la pérdida de prueba
        self.log('test_loss', loss)
        return loss
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        return optimizer

@hydra.main(config_path=".", config_name="config")
def main(cfg: DictConfig):
    model = LitAutoencoder(cfg)
    trainer = pl.Trainer(max_epochs=cfg.trainer.max_epochs)
    trainer.fit(model)

if __name__ == '__main__':
    main()